<a href="https://colab.research.google.com/github/slilli23/Horcynus-orca-stile/blob/main/Horcynus_Orca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Esplorare l'anomalia: uno studio computazionale dello stile di Horcinus Orca
di (anonimizzato)

Il contributo propone un approccio innovativo allo studio linguistico di Horcynus Orca, integrando lettura ravvicinata e strumenti della linguistica computazionale. L’esplorazione del testo tramite la codifica di espressioni regolari (REGEX) consente di superare l’ostacolo costituito dall’estensione e complessità del romanzo – che ha spesso imposto un approccio esemplificativo – e incoraggia una formalizzazione in linea con la «grammaticalizzazione» dell’elemento deformante riconosciuta all’autore. L’analisi si concentra, a titolo esemplificativo, su due strategie inventive: le formazioni verbali parasintetiche riflessive e i reduplicati con funzione elativa. Il confronto con repertori esistenti ha rivelato nuove occorrenze riconducibili alla creatività autoriale, confermando che l’approccio integrato tra *distant* e *close reading* garantisce un’analisi più precisa e completa, aprendo nuove possibilità interpretative.

Definizione delle REGEX per identificare i fenomeni di interesse nel testo:

*   Formazioni verbali parasintetiche riflessive con prefisso in A(D)-

    **\ba([bcdfglmnpqrstvz])\1[a-zà-ù]{4,}[ro]si\b**

    **\b(s[’']a([bcdfglmnpqrstvz])\2[a-zà-ù]{4,})**


*   Reduplicati con funzione elativa

    **\b(\w+)[\s]?\1\b**

Si procede quindi all'importazione del testo in formato .txt e all'estrazione dei risultati attraverso l'applicazione delle REGEX.




# Setup

In [ ]:
# Clona il la cartella dal repository GitHub
REPO_URL  = #anonimizzato per peer review"
REPO_NAME = "Horcynus-orca-stile"

from pathlib import Path

# Torna alla radice di Colab per sicurezza
%cd /content

if (Path(REPO_NAME) / ".git").exists():
    %cd {REPO_NAME}
    !git pull --ff-only
else:
    !git clone --depth=1 {REPO_URL}
    %cd {REPO_NAME}

# Importa le librerie necessarie per l'analisi
from pathlib import Path
import re
import pandas as pd
import csv
from collections import Counter
from collections import defaultdict


# Definisci le cartelle
ROOT = Path.cwd()
DATA_DIR = ROOT / "data"
SCRIPTS_DIR = ROOT / "scripts"
RESULTS_DIR = ROOT / "results"

#Verifica l'esistenza delle cartelle e la correttezza dei percorsi
RESULTS_DIR.mkdir(exist_ok=True)

print("Root:", ROOT)
print("Data:", DATA_DIR)
print("Scripts:", SCRIPTS_DIR)
print("Results:", RESULTS_DIR)

# Carica il testo
file_path = DATA_DIR / "Horcynus_orca.txt"

with open(file_path, encoding="utf-8") as f:
    text = f.read()

print("Lunghezza testo:", len(text), "caratteri")
print("Primi 500 caratteri:\n", text[:500])


# Parasintetici con prefisso in A(D)-

Applichiamo la prima REGEX (verbi parasintetici riflessivi con prefisso in ad-, forma indefinita).

In [ ]:
# Imposta la cartella di output
OUTPUT_DIR = Path.cwd() / "results"
OUTPUT_DIR.mkdir(exist_ok=True)

# Salva la REGEX in una variabile (raw string per non dover raddoppiare i backslash)
pattern = r"\ba([bcdfglmnpqrstvz])\1[a-zà-ù]{4,}[ro]si\b"
rx = re.compile(pattern, flags=re.IGNORECASE | re.UNICODE)

# Applica la REGEX e salva il match con il con contesto (5 parole prima e dopo)
words = text.split()
matches = []

TRAIL_PUNCT_RE = re.compile(r"[.,;:!?…]+$")  # punteggiatura finale

for i, word in enumerate(words):
    # 1) rimuovi punteggiatura finale
    token = TRAIL_PUNCT_RE.sub("", word)

    # 2) separa per apostrofo
    parts = re.split(r"[’']", token)

    found_for_token = False
    for part in parts:
        if not part:
            continue
        if rx.search(part):
            left_context = " ".join(words[max(0, i-5):i])
            right_context = " ".join(words[i+1:i+6])
            # salva SOLO la parte dopo l'apostrofo che ha matchato
            matches.append([part, left_context, right_context])
            found_for_token = True
            break  # evita duplicati se la stessa parola produce più sottoparti

# Definisci il file di output
OUTPUT_PATH = OUTPUT_DIR / "parasintetici_indefiniti.csv"

# Salva i risultati in CSV
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

with open(OUTPUT_PATH, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["match", "left_context", "right_context"])
    writer.writerows(matches)

print(f"Salvati {len(matches)} risultati in {OUTPUT_PATH}")


Verifichiamo se le parole trovate appartengono al vocabolario italiano, utilizzando la libreria spaCy. Per prima cosa scarichiamo il modello utilizzato da spaCy. Viene scelto il modello 'large', per ottenere una prestazione più accurata.

In [ ]:
# Installazione
!pip -q install -U pip setuptools wheel
!pip -q install "numpy==1.26.4" "spacy==3.7.5"
!python -m spacy download it_core_news_lg


# --- Setup ---
import spacy

Riavviamo il runtime per risolvere i problemi di dipendenze (Ctrl+M).

In [ ]:
# --- Setup ---
import spacy

Annotiamo il testo con la libreria spaCy.

In [ ]:
ROOT = Path.cwd().resolve()
IN_PATH  = ROOT / "results" / "parasintetici_indefiniti.csv"
OUT_PATH = ROOT / "results" / "parasintetici_indefiniti_spacy.csv"
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)

nlp = spacy.load("it_core_news_lg")

TRAIL_PUNCT_RE = re.compile(r"[.,;:!?…]+$")

def variants(w: str):
    w0 = w.strip()
    w0 = TRAIL_PUNCT_RE.sub("", w0)
    outs = set([w0, w0.lower(), w0.replace("’","'")])
    # se riflessivo in -rsi prova senza 'si' (infinito)
    wl = w0.lower().replace("’","'")
    if wl.endswith("rsi"):
        outs.add(wl[:-2])
    # rimuovi apostrofo iniziale di clitici frequenti
    for pref in ("l'", "s'", "m'", "t'", "d'", "n'"):
        if wl.startswith(pref):
            outs.add(wl[len(pref):])
    return outs

def spacy_check(word: str):
    # ordina preferendo la forma originale e la minuscola
    cand = sorted(list(variants(word)), key=lambda x: (x != word, x != word.lower()))
    known = False
    chosen = None
    for v in cand:
        if not nlp.vocab[v].is_oov:
            known = True
            chosen = v
            break
    form = chosen or cand[0]
    doc = nlp(form)
     # usa il primo token "parola" (salta eventuali segni)
    tok = next((t for t in doc if t.is_alpha or t.text.strip("'")), doc[0])
    return known, form, tok.lemma_

rows_out = []
with open(IN_PATH, "r", encoding="utf-8") as f:
    r = csv.DictReader(f)
    for row in r:
        w = row["match"]
        known, chosen, lemma = spacy_check(w)
        row["spacy_known"]   = known
        row["chosen_variant"] = chosen    # forma su cui è stato fatto il check
        row["lemma"]          = lemma
        rows_out.append(row)

# Scrivi output
with open(OUT_PATH, "w", newline="", encoding="utf-8") as f:
    fieldnames = list(rows_out[0].keys())
    w = csv.DictWriter(f, fieldnames=fieldnames)
    w.writeheader()
    w.writerows(rows_out)

print(f"Annotate {len(rows_out)} forme. File scritto in: {OUT_PATH}")



Accorpiamo le voci in types distinguendo le voci omografe che hanno avuto diverse etichette generate da spaCy.

In [ ]:
# Conta per (type, spacy_known)
counter = defaultdict(int)

for row in rows_out:
    type_form = row["match"].strip().lower()
    spacy_status = row["spacy_known"]
    counter[(type_form, spacy_status)] += 1

# Crea DataFrame
aggregated = []
for (type_form, spacy_status), freq in sorted(counter.items()):
    aggregated.append({
        "type": type_form,
        "spacy_known": spacy_status,
        "freq": freq
    })

df_out = pd.DataFrame(aggregated)
df_out = df_out.sort_values(by=["freq", "type"], ascending=[False, True]).reset_index(drop=True)

# --- Salva
AGG_OUT_PATH = ROOT / "results" / "parasintetici_indefiniti_types_spacy.csv"
df_out.to_csv(AGG_OUT_PATH, index=False, encoding="utf-8")

print(f"Salvati {len(df_out)} types distinti con spacy_known in {AGG_OUT_PATH}")


Applichiamo la seconda REGEX (forme parasintetiche riflessive con prefisso A(D)- nelle forme finite del verbo.

In [ ]:
# Imposta la cartella di output
OUTPUT_DIR = Path.cwd() / "results"
OUTPUT_DIR.mkdir(exist_ok=True)

# Salva la REGEX in una variabile (raw string per non dover raddoppiare i backslash)
pattern = r"\b(s[’']a([bcdfglmnpqrstvz])\2[a-zà-ù]{4,})"
rx = re.compile(pattern, flags=re.IGNORECASE | re.UNICODE)

# Applica la REGEX e salva il match con il con contesto (5 parole prima e dopo)
words = text.split()
matches = []

for i, word in enumerate(words):
    if rx.search(word):
        left_context = " ".join(words[max(0, i-5):i])
        right_context = " ".join(words[i+1:i+6])
        matches.append([word, left_context, right_context])

# Definisci il file di output
OUTPUT_PATH = OUTPUT_DIR / "parasintetici_finiti.csv"

# Salva i risultati in CSV
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

with open(OUTPUT_PATH, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["match", "left_context", "right_context"])
    writer.writerows(matches)

print(f"Salvati {len(matches)} risultati in {OUTPUT_PATH}")



Annotiamo il testo attraverso spaCy per verificare le forme riconosciute come italiane.

In [ ]:
ROOT = Path.cwd().resolve()
IN_PATH  = ROOT / "results" / "parasintetici_finiti.csv"
OUT_PATH = ROOT / "results" / "parasintetici_finiti_spacy.csv"
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)

nlp = spacy.load("it_core_news_lg")

_APOS = r"[’']"  # apostrofo semplice o tipografico
S_PREFIX_RE = re.compile(rf"^\s*s{_APOS}\s*(.+)\s*$", flags=re.IGNORECASE)
TRAIL_PUNCT_RE = re.compile(r"[.,;:!?…]+$")

# considera solo la parola dopo la sequenza s'
def extract_verb_form(w: str) -> str:
    w0 = w.strip()
    w0 = TRAIL_PUNCT_RE.sub("", w0)
    m = S_PREFIX_RE.match(w0)
    if m:
        return m.group(1).strip()
    return w0

def variants(w: str):
    w0 = extract_verb_form(w).strip()
    return {w0, w0.lower()}


def spacy_check(word: str):

    cand = sorted(list(variants(word)), key=lambda x: (x != word, x != word.lower()))
    known = False
    chosen = None
    for v in cand:
        if not nlp.vocab[v].is_oov:
            known = True
            chosen = v
            break
    form = chosen or cand[0]
    doc = nlp(form)
    tok = next((t for t in doc if t.is_alpha or t.text.strip("'")), doc[0])
    return known, form, tok.lemma_

rows_out = []
with open(IN_PATH, "r", encoding="utf-8") as f:
    r = csv.DictReader(f)
    for row in r:
        w = row["match"]
        known, chosen, lemma = spacy_check(w)
        row["spacy_known"]   = known
        row["chosen_variant"] = chosen    # forma su cui è stato fatto il check
        row["lemma"]          = lemma
        rows_out.append(row)

# Scrivi output
with open(OUT_PATH, "w", newline="", encoding="utf-8") as f:
    fieldnames = list(rows_out[0].keys())
    w = csv.DictWriter(f, fieldnames=fieldnames)
    w.writeheader()
    w.writerows(rows_out)

print(f"Annotate {len(rows_out)} forme. File scritto in: {OUT_PATH}")



Accorpiamo le voci in types distinguendo le voci omografe che hanno avuto diverse etichette generate da spaCy.

In [ ]:
# 1. Normalizza e aggrega
counter = defaultdict(int)  # chiave = (type, spacy_known), valore = frequenza

for row in rows_out:
    type_form = extract_verb_form(row["match"]).lower().strip()
    spacy_status = row["spacy_known"]
    counter[(type_form, spacy_status)] += 1

# 2. Crea dataframe
aggregated = []
for (type_form, spacy_status), freq in sorted(counter.items()):
    aggregated.append({
        "type": type_form,
        "spacy_known": spacy_status,
        "freq": freq
    })

df_out = pd.DataFrame(aggregated)
df_out = df_out.sort_values(by=["freq", "type"], ascending=[False, True]).reset_index(drop=True)

# 3. Salva CSV
AGG_OUT_PATH = ROOT / "results" / "parasintetici_finiti_types_spacy.csv"
df_out.to_csv(AGG_OUT_PATH, index=False, encoding="utf-8")

print(f"Salvati {len(df_out)} types distinti con spacy_known in {AGG_OUT_PATH}")

Applichiamo la REGEX per trovare nel testo anche le sequenze con "si" non eliso.

In [ ]:
import re, csv
from pathlib import Path

# --- Input: il testo intero in una stringa ---
# text = ...

# Imposta la cartella di output
OUTPUT_DIR = Path.cwd() / "results"
OUTPUT_DIR.mkdir(exist_ok=True)

# REGEX per la seconda parola: a + C + C + almeno 4 lettere
AWORD_RE = re.compile(r"^a([bcdfglmnpqrstvz])\1[a-zà-ù]{4,}$", flags=re.IGNORECASE | re.UNICODE)

# Pulisci punteggiatura iniziale/finale e normalizza apostrofi
TRAIL_PUNCT_RE = re.compile(r"[.,;:!?…)\]»”]+$")
LEAD_PUNCT_RE  = re.compile(r"^[([«“\"(]+")

def clean_token(w: str) -> str:
    if w is None:
        return ""
    w = str(w)
    w = LEAD_PUNCT_RE.sub("", w)
    w = TRAIL_PUNCT_RE.sub("", w)
    return w.replace("’", "'").strip()

# Tokenizza in modo semplice (per contesti bastano gli spazi)
words = text.split()
matches = []

i = 0
while i < len(words) - 1:
    w0_raw = words[i]
    w1_raw = words[i+1]

    w0 = clean_token(w0_raw)
    w1 = clean_token(w1_raw)

    # cerchiamo "si" come parola intera (case-insensitive)
    if w0.lower() == "si" and AWORD_RE.match(w1):
        left_context  = " ".join(words[max(0, i-5):i])
        # >>> salva SOLO la seconda parola (pulita) <<<
        match_text    = w1
        right_context = " ".join(words[i+2:i+7])
        matches.append([match_text, left_context, right_context])
        i += 2
        continue

    i += 1

# Definisci il file di output
OUTPUT_PATH = OUTPUT_DIR / "parasintetici_finiti_si.csv"
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

# Salva i risultati in CSV
with open(OUTPUT_PATH, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["match", "left_context", "right_context"])
    writer.writerows(matches)

print(f"Salvati {len(matches)} risultati in {OUTPUT_PATH}")

Annotiamo i risultati con la libreria spaCy.

In [ ]:
ROOT = Path.cwd().resolve()
IN_PATH  = ROOT / "results" / "parasintetici_finiti_si.csv"
OUT_PATH = ROOT / "results" / "parasintetici_finiti_si_spacy.csv"
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)

nlp = spacy.load("it_core_news_lg")

def variants(w: str):
    w0 = w.strip()
    return {w0, w0.lower()}

def spacy_check(word: str):
    cand = sorted(list(variants(word)), key=lambda x: (x != word, x != word.lower()))
    known = False
    chosen = None
    for v in cand:
        if not nlp.vocab[v].is_oov:
            known = True
            chosen = v
            break
    form = chosen or cand[0]
    doc = nlp(form)
    tok = next((t for t in doc if t.is_alpha or t.text.strip("'")), doc[0])
    return known, form, tok.lemma_

rows_out = []
with open(IN_PATH, "r", encoding="utf-8") as f:
    r = csv.DictReader(f)
    for row in r:
        w = row["match"]
        known, chosen, lemma = spacy_check(w)
        row["spacy_known"]    = known
        row["chosen_variant"] = chosen
        row["lemma"]          = lemma
        rows_out.append(row)

with open(OUT_PATH, "w", newline="", encoding="utf-8") as f:
    fieldnames = list(rows_out[0].keys())
    w = csv.DictWriter(f, fieldnames=fieldnames)
    w.writeheader()
    w.writerows(rows_out)

print(f"Annotate {len(rows_out)} forme. File scritto in: {OUT_PATH}")


Accorpiamo le voci in types distinguendo le voci omografe che hanno avuto diverse etichette generate da spaCy.

In [ ]:
# 1. Normalizza e aggrega
counter = defaultdict(int)  # chiave = (type, spacy_known), valore = frequenza

for row in rows_out:
    type_form = extract_verb_form(row["match"]).lower().strip()
    spacy_status = row["spacy_known"]
    counter[(type_form, spacy_status)] += 1

# 2. Crea dataframe
aggregated = []
for (type_form, spacy_status), freq in sorted(counter.items()):
    aggregated.append({
        "type": type_form,
        "spacy_known": spacy_status,
        "freq": freq
    })

df_out = pd.DataFrame(aggregated)
df_out = df_out.sort_values(by=["freq", "type"], ascending=[False, True]).reset_index(drop=True)

# 3. Salva CSV
AGG_OUT_PATH = ROOT / "results" / "parasintetici_finiti_si_types_spacy.csv"
df_out.to_csv(AGG_OUT_PATH, index=False, encoding="utf-8")

print(f"Salvati {len(df_out)} types distinti con spacy_known in {AGG_OUT_PATH}")

Unifichiamo i file generati con le voci raggruppate per types.

In [ ]:
ROOT = Path.cwd().resolve()
FINITI_PATH = ROOT / "results" / "parasintetici_finiti_types_spacy.csv"
INDEFINITI_PATH = ROOT / "results" / "parasintetici_indefiniti_types_spacy.csv"
FINITI_SI_PATH = ROOT / "results" / "parasintetici_finiti_si_types_spacy.csv"
OUT_PATH = ROOT / "results" / "parasintetici_types_spacy_unificato.csv"

# Carica
df_finiti = pd.read_csv(FINITI_PATH)
df_indefiniti = pd.read_csv(INDEFINITI_PATH)
df_finiti_si = pd.read_csv(FINITI_SI_PATH)

# Unisci
df_all = pd.concat([df_finiti, df_indefiniti, df_finiti_si], ignore_index=True)

# Dizionario di aggregazione minimo per le tue colonne
agg_dict = {
    "spacy_known": "max",   # True se almeno una volta
    "freq": "sum"           # somma delle frequenze su tutte le categorie
}

# Unifica per 'type'
df_all_unique = (
    df_all.groupby("type", as_index=False)
          .agg(agg_dict)
          .sort_values("type")
          .reset_index(drop=True)
)

# Salva
df_all_unique.to_csv(OUT_PATH, index=False, encoding="utf-8")

print(f"File unificato scritto in: {OUT_PATH}")
print(f"Totale types unici: {len(df_all_unique)}")


Calcoliamo ora le misure di precisione e recall per la capacità di spaCy di catturare veri negativi. La correzione manuale è stata fatta basando il confronto sulla presenza della parola nel GDLI (Battaglia). I risultati sono i seguenti (i valori fanno riferimento alle occorrenze dei tokens):

1.   Forme non riconosciute (TRUE) = 88
2.   Forme non riconosciute (FALSE) = 187
3.   Forme riconosciute (TRUE) = 524
4.   Forme riconosciute (FALSE) = 2

Poiché l'interesse è nel valutare la capacità di individuare correttamente i true negatives, vengono invertite le classi di positività/negatività, in modo che 1 = TP, 2 = FP, 3 = TN, 4 = FN.  



In [ ]:
# Dati forniti
FN = 187
FP = 2
TN = 88
TP = 524
TOTAL = FN + FP + TN + TP

# === Inversione delle classi ===
# Ora consideriamo come "positivi" i TN (prima negativi)
# e come "negativi" i TP (prima positivi).

# Rinominiamo i valori dopo l'inversione
TP_inv = TN   # veri positivi diventano i vecchi TN
TN_inv = TP   # veri negativi diventano i vecchi TP
FP_inv = FN   # falsi positivi diventano i vecchi FN
FN_inv = FP   # falsi negativi diventano i vecchi FP

# Calcoli standard
precision = TP_inv / (TP_inv + FP_inv)
recall = TP_inv / (TP_inv + FN_inv)
f1 = 2 * (precision * recall) / (precision + recall)

print("=== Risultati con classi invertite ===")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1-score:  {f1:.3f}")


I risultati dimostrano l'affidabilità di spaCy nel catturare tutte le occorrenze di veri negativi (97,8%), ma scarsa precisione (32%), dati i numerosi casi di falsi negativi (quasi esclusivamente nelle forme finite). Poiché interessa una scrematura iniziale preliminare al controllo manuale, il metodo risulta affidabile, anche se testare altri metodi potrebbe portare a risultati ancora più efficienti.  

Ora creiamo una lista con i risultati identificati come veri negativi, lemmatizzati alla forma dell'infinito. (Il file con i true negatives è caricato manualmente nella cartella 'results').

A questo punto verifichiamo quali types identificati non sono ricompresi nell'"Onomaturgia darrighiana" di G. Alvino (2012), e quali invece si trovano in Alvino e non sono stati catturati dal nostro codice. Poiché le nostre forme non sono state lemmatizzate, il confronto viene fatto sulla corrispondenza dei primi 4 caratteri

In [ ]:
ROOT = Path.cwd().resolve()
VERI_NEGATIVI_TXT = ROOT / "results" / "veri_negativi.txt"
ALVINO_TXT       = ROOT / "data" / "onomaturgia_alvino_parasint.txt"

OUT1 = ROOT / "results" / "parasintetici_solo_in_veri_negativi.csv"
OUT2 = ROOT / "results" / "parasintetici_solo_in_alvino.csv"

def load_types_txt(path: Path) -> set[str]:
    """Carica un file .txt con una voce per riga"""
    types = set()
    with open(path, encoding="utf-8") as f:
        for line in f:
            t = line.strip()
            if not t:
                continue
            types.add(t.lower())
    return types

# --- Carica i dati (entrambi i file sono .txt)
csv_types    = load_types_txt(VERI_NEGATIVI_TXT)
alvino_types = load_types_txt(ALVINO_TXT)

# --- Differenze
solo_csv    = csv_types - alvino_types
solo_alvino = alvino_types - csv_types

# --- Salva risultati
OUT1.parent.mkdir(parents=True, exist_ok=True)
pd.DataFrame({"type": sorted(solo_csv)}).to_csv(OUT1, index=False, encoding="utf-8")
pd.DataFrame({"type": sorted(solo_alvino)}).to_csv(OUT2, index=False, encoding="utf-8")

print(f"Salvati {len(solo_csv)} types solo in veri_negativi -> {OUT1}")
print(f"Salvati {len(solo_alvino)} types solo in onomaturgia -> {OUT2}")



# Reduplicati con funzione elativa

Proseguiamo ora con la ricerca dei reduplicati con funzione elativa. Applichiamo la REGEX al testo per trovare le forme reduplicate univerbate, e quindi le forme reduplicate con spazio.

In [ ]:
# Imposta la cartella di output
OUTPUT_DIR = Path.cwd() / "results"
OUTPUT_DIR.mkdir(exist_ok=True)

# Regex per riduplicazioni:
# 1) identiche con o senza spazio (già esistente)
pattern_exact = re.compile(r"\b(\w+)[\s]?\1\b", flags=re.IGNORECASE)

# 2) univerbate con vocale fusa: c r c r c (c = vocale, r = parte centrale)
pattern_fused = re.compile(r"\b([aeiou])(\w+)\1\2\1\b", flags=re.IGNORECASE)

# Tokenizzazione semplice per estrarre contesto
words = re.findall(r"\w+|[^\w\s]", text, flags=re.UNICODE)
matches = []

def clean_forma(s: str) -> str:
    return s.rstrip(".,;:!?…").lower()

for i in range(1, len(words) - 1):
    word = words[i].lower()
    prev_word = words[i-1].lower()
    next_word = words[i+1].lower()

    # --- UNIVERBATE ---
    if re.fullmatch(pattern_exact, word) or re.fullmatch(pattern_fused, word):
        forma = clean_forma(word)
        if len(forma) >= 6:  # minimo 3+3
            matches.append({
                "forma": forma,
                "contesto": " ".join(words[max(0, i-5):i+6]),
                "tipo": "univerbata"
            })

    # --- CON SPAZIO ---
    bigram = f"{prev_word} {word}"
    if re.fullmatch(pattern_exact, bigram):
        forma = clean_forma(bigram)
        if len(forma.replace(" ", "")) >= 6:  # 3+3 senza contare lo spazio
            matches.append({
                "forma": forma,
                "contesto": " ".join(words[max(0, i-6):i+5]),
                "tipo": "con spazio"
            })

# Percorso file output
OUT_PATH = OUTPUT_DIR / "forme_reduplicate_con_contesto.csv"

# Scrivi il CSV
with open(OUT_PATH, mode="w", encoding="utf-8", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["forma", "tipo", "contesto"])
    writer.writeheader()
    for row in matches:
        writer.writerow(row)

print(f"File salvato in: {OUT_PATH}")



Ora estraiamo solo le forme univerbate e raggruppiamole in types.

In [ ]:
# Percorso file input creato dallo script precedente
INPUT_PATH = Path.cwd() / "results" / "forme_reduplicate_con_contesto.csv"
OUTPUT_PATH = Path.cwd() / "results" / "forme_univerbate_frequenze.csv"

# Carica il CSV
df = pd.read_csv(INPUT_PATH)

# Filtra solo le forme univerbate
df_univerbate = df[df["tipo"] == "univerbata"].copy()

# Normalizza la forma (minuscole, rimuovi punteggiatura finale se presente)
df_univerbate["forma"] = df_univerbate["forma"].str.lower().str.replace(r"[.,;:!?…]+$", "", regex=True)

# Conta i types e le frequenze
frequenze = df_univerbate["forma"].value_counts().reset_index()
frequenze.columns = ["forma", "frequenza"]

# Salva il CSV
frequenze.to_csv(OUTPUT_PATH, index=False)
print(f"File salvato in: {OUTPUT_PATH}")



Ora confrontiamo le voci trovate con quelle dell'*Onomaturgia darrighiana* di Alvino (2012).

In [ ]:
ROOT = Path.cwd().resolve()
ALVINO_TXT = ROOT / "data" / "onomaturgia_alvino_reduplicati.txt"
OUT_SOLO_ALVINO = ROOT / "results" / "reduplicati_solo_in_alvino.csv"
OUT_SOLO_UNIVERBATE = ROOT / "results" / "reduplicati_solo_in_univerbate.csv"

# CARICA FREQUENZE (già creato sopra come OUTPUT_PATH)
freq_df = pd.read_csv(OUTPUT_PATH)

# CARICA LISTA ALVINO
TRAIL_PUNCT_RE = re.compile(r"[.,;:!?…]+$")

def load_types_txt(path: Path) -> set[str]:
    types = set()
    with open(path, encoding="utf-8") as f:
        for line in f:
            t = line.strip()
            if not t:
                continue
            if t:
                types.add(t)
    return types

alvino_set = load_types_txt(ALVINO_TXT)
univerbate_set = set(freq_df["forma"].tolist())

# DIFFERENZE
solo_in_alvino = sorted(alvino_set - univerbate_set)
solo_in_univerbate = univerbate_set - alvino_set

# SALVA FILES
# 1) Solo in Alvino (senza frequenze)
pd.DataFrame({"forma": solo_in_alvino}).to_csv(OUT_SOLO_ALVINO, index=False)

# 2) Solo in forme_univerbate (con frequenze)
df_solo_univerbate = freq_df[freq_df["forma"].isin(solo_in_univerbate)].copy()
df_solo_univerbate.sort_values(["frequenza", "forma"], ascending=[False, True], inplace=True)
df_solo_univerbate.to_csv(OUT_SOLO_UNIVERBATE, index=False)

print(f"File salvati:\n- Solo in Alvino: {OUT_SOLO_ALVINO}\n- Solo in forme_univerbate: {OUT_SOLO_UNIVERBATE}")

Nella cartella 'results' della repository sulla piattaforma GitHub sono caricati i file di confronto annotati con le spiegazioni sul mancato match.